In [66]:
from langchain.chains.summarize import load_summarize_chain
from langchain.chains.openai_functions import create_structured_output_runnable
from langchain.document_loaders import PyPDFLoader
from langchain import OpenAI, PromptTemplate
import PyPDF2
import glob
import os
from dotenv import load_dotenv

# %load_ext dotenv
# %dotenv ./.env
config = load_dotenv('.env')
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

In [22]:
def extract_pdf_text(filename):
    file = open(filename, 'rb')
    reader = PyPDF2.PdfReader(file)
    text = ''
    for page in range(len(reader.pages)):
        page = reader.pages[page]
        text += page.extract_text()
    return text

In [14]:
def extract_pdf_text_from_dir(dir):
    text = ''
    for filename in glob.glob(os.path.join(dir, '*.pdf')):
        text += extract_pdf_text(filename)
    return text

In [28]:
# text = extract_pdf_text_from_dir(os.path.join('./data', 'pdfs'))
with open('data/exams.txt', 'w') as f:
    f.write(text)


In [35]:
text = extract_pdf_text('data/pdfs/exam.pdf')

In [24]:
llm = OpenAI(temperature=0.2, openai_api_key=OPENAI_API_KEY, model='gpt-3.5-turbo')

In [32]:
schema = {
    "properties": {
        "question_id": {"type": "string"},
        "question": {"type": "string"},
        "choices": {"type": "array", "option": {"type": "string"}},
        "answer": {"type": "string"},
        "question_type": {"type": "string"},
        "topic": {"type": "string"},
    },
    "required": ["question_id", "question", "answer", "question_type"]
}

In [70]:
def extract_question_answer_data(doc, schema, llm):

    
    # input = prompt.format_prompt(exam_text=doc, schema=schema)
    runnable = create_structured_output_runnable(schema, llm, prompt)
    extract_output = runnable.invoke({'exam_text': "what is the capital of France? \n\n Paris"})
    return extract_output

In [68]:
loader = PyPDFLoader('data/pdfs/exam.pdf')
docs = loader.load_and_split()
print(len(docs))
print(docs[3])

25
page_content='CS230\nSolution: (iii)\n(e)(1 point) Consider the model deﬁned in question (d) with parameters initialized with\nzeros.W[1]denotes the weight matrix of the ﬁrst layer. You forward propagate a batch\nof examples, and then backpropagate the gradients and update the parameters. Which\nof the following statements is true?\n(i) Entries of W[1]may be positive or negative\n(ii) Entries of W[1]are all negative\n(iii) Entries of W[1]are all positive\n(iv) Entries of W[1]are all zeros\nSolution: (i)\n(f)(2 points) Consider the layers landl−1 in a fully connected neural network:\nThe forward propagation equations for these layers are:\nz[l−1]=W[l−1]a[l−2]+b[l−1]\na[l−1]=g[l−1](z[l−1])\nz[l]=W[l]a[l−1]+b[l]\na[l]=g[l](z[l])\nWhich of the following propositions is true? Xavier initialization ensures that :\n(i)Var(W[l−1]) is the same as Var(W[l]).\n(ii)Var(b[l]) is the same as Var(b[l−1]).\n(iii)Var(a[l]) is the same as Var(a[l−1]), at the end of training.\n(iv)Var(a[l]) is the sam

In [73]:
prompt = PromptTemplate.from_template(
    "Read the following exam paper and extract the information requested in the schema for each question \n\n{exam_text}\n\n\n\nSchema: {schema}\n\n\n\n"
)

In [74]:
runnable = create_structured_output_runnable(schema, llm, prompt)
prompt.format(exam_text=docs[3], schema=schema)

"Read the following exam paper and extract the information requested in the schema for each question \n\npage_content='CS230\\nSolution: (iii)\\n(e)(1 point) Consider the model deﬁned in question (d) with parameters initialized with\\nzeros.W[1]denotes the weight matrix of the ﬁrst layer. You forward propagate a batch\\nof examples, and then backpropagate the gradients and update the parameters. Which\\nof the following statements is true?\\n(i) Entries of W[1]may be positive or negative\\n(ii) Entries of W[1]are all negative\\n(iii) Entries of W[1]are all positive\\n(iv) Entries of W[1]are all zeros\\nSolution: (i)\\n(f)(2 points) Consider the layers landl−1 in a fully connected neural network:\\nThe forward propagation equations for these layers are:\\nz[l−1]=W[l−1]a[l−2]+b[l−1]\\na[l−1]=g[l−1](z[l−1])\\nz[l]=W[l]a[l−1]+b[l]\\na[l]=g[l](z[l])\\nWhich of the following propositions is true? Xavier initialization ensures that :\\n(i)Var(W[l−1]) is the same as Var(W[l]).\\n(ii)Var(b[l]) 

In [71]:
questions_json = extract_question_answer_data(docs[3], schema, llm)
with open('data/exams.json', 'w') as f:
    f.write(questions_json)

KeyError: 'schema'